In [3]:
import cv2
from cvzone.HandTrackingModule import HandDetector
import numpy as np
import math
import time
import os

In [5]:
import cv2
import os
import numpy as np
import math
import time
from cvzone.HandTrackingModule import HandDetector
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, array_to_img

# Webcam & hand detector
cap = cv2.VideoCapture(0)
detector = HandDetector(maxHands=1)

# Image setup
offset = 20
imgSize = 300

# Input label name
label = input("Enter label name: ")

# Create folders
os.makedirs('data', exist_ok=True)
os.makedirs('augmented', exist_ok=True)
data_folder = f'data/{label}'
aug_folder = f'augmented/{label}'
os.makedirs(data_folder, exist_ok=True)
os.makedirs(aug_folder, exist_ok=True)

counter = 0

# Image augmenter
augmentor = ImageDataGenerator(
    rotation_range=15,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    brightness_range=[0.8, 1.2],
    horizontal_flip=True,
    fill_mode='nearest'
)

while True:
    success, img = cap.read()
    hands, img = detector.findHands(img)

    if hands:
        hand = hands[0]
        x, y, w, h = hand['bbox']

        imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255

        y1 = max(0, y - offset)
        y2 = min(img.shape[0], y + h + offset)
        x1 = max(0, x - offset)
        x2 = min(img.shape[1], x + w + offset)

        imgCrop = img[y1:y2, x1:x2]
        aspectRatio = h / w

        if aspectRatio > 1:
            k = imgSize / h
            wCal = math.ceil(k * w)
            imgResize = cv2.resize(imgCrop, (wCal, imgSize))
            wGap = math.ceil((imgSize - wCal) / 2)
            imgWhite[:, wGap:wGap + wCal] = imgResize
        else:
            k = imgSize / w
            hCal = math.ceil(k * h)
            imgResize = cv2.resize(imgCrop, (imgSize, hCal))
            hGap = math.ceil((imgSize - hCal) / 2)
            imgWhite[hGap:hGap + hCal, :] = imgResize

        cv2.imshow("ImageCrop", imgCrop)
        cv2.imshow("ImageWhite", imgWhite)

    cv2.imshow("Image", img)
    key = cv2.waitKey(1)

    if key == ord("s"):
        # Save original
        filename = f'Image_{time.time()}.jpg'
        path = os.path.join(data_folder, filename)
        cv2.imwrite(path, imgWhite)
        print(f"Saved original image {counter + 1} to {path}")

        # Augment 9 times and save
        x = img_to_array(imgWhite)
        x = x.reshape((1,) + x.shape)
        i = 0
        for batch in augmentor.flow(x, batch_size=1,
                                     save_to_dir=aug_folder,
                                     save_prefix='aug',
                                     save_format='jpg'):
            i += 1
            if i >= 9:
                break
        counter += 1
        print(f"Augmented 9 images for {label} (Total sets: {counter})")

    if key == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


Saved original image 1 to data/Image_1749458881.5508854.jpg
Augmented 9 images for  (Total sets: 1)
Saved original image 2 to data/Image_1749458884.144751.jpg
Augmented 9 images for  (Total sets: 2)
Saved original image 3 to data/Image_1749458887.7692177.jpg
Augmented 9 images for  (Total sets: 3)
Saved original image 4 to data/Image_1749458890.1341186.jpg
Augmented 9 images for  (Total sets: 4)
Saved original image 5 to data/Image_1749458894.3461792.jpg
Augmented 9 images for  (Total sets: 5)
Saved original image 6 to data/Image_1749458897.934945.jpg
Augmented 9 images for  (Total sets: 6)
Saved original image 7 to data/Image_1749458900.3918154.jpg
Augmented 9 images for  (Total sets: 7)
Saved original image 8 to data/Image_1749458904.7467482.jpg
Augmented 9 images for  (Total sets: 8)


In [7]:
# ✅ 1. Imports
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# ✅ 2. Parameters
img_size = 300
batch_size = 32
epochs = 10
data_path = 'augmented'  # your folder path like augmented/A, augmented/B etc.

# ✅ 3. Data Preparation
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_data = datagen.flow_from_directory(
    data_path,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_data = datagen.flow_from_directory(
    data_path,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

num_classes = len(train_data.class_indices)
print(f"✅ Number of classes: {num_classes}")

# ✅ 4. Model Creation
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(img_size, img_size, 3)),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# ✅ 5. Compile Model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# ✅ 6. Train the Model
model.fit(train_data, epochs=epochs, validation_data=val_data)

# ✅ 7. Save the Model
model.save("sign_language_model.h5")
print("✅ Model saved as sign_language_model.h5")


Found 361 images belonging to 3 classes.
Found 89 images belonging to 3 classes.
✅ Number of classes: 3


c:\Users\Tharun B S\Documents\VS Code\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\Tharun B S\Documents\VS Code\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step - accuracy: 0.3845 - loss: 12.4226 - val_accuracy: 0.8315 - val_loss: 0.4944
Epoch 2/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.8714 - loss: 0.3887 - val_accuracy: 0.8427 - val_loss: 0.3170
Epoch 3/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.9597 - loss: 0.1563 - val_accuracy: 0.8427 - val_loss: 0.4477
Epoch 4/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.9813 - loss: 0.0583 - val_accuracy: 0.8315 - val_loss: 0.6880
Epoch 5/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.9829 - loss: 0.0478 - val_accuracy: 0.8090 - val_loss: 1.0630
Epoch 6/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.9829 - loss: 0.0375 - val_accuracy: 0.8539 - val_loss: 1.0389
Epoch 7/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.9925 - loss: 0.0298 - val_accuracy: 0.8427 - val_loss: 0.4839
Epoch 8/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 1.0000 - loss: 0.0142 - val_accuracy: 0.8427 - val_loss

✅ Model saved as sign_language_model.h5
